# Question 1

# Q 2.2

### Data collection

In [1]:
import requests
import json
import pandas as pd
import datetime
from retry import retry
import time

In [2]:
@retry(tries=3, delay=5)
def getDataAtTimestamp(year, month, day, hour, min_str='01', sec_str = '00'):
    try:
        site = f'https://api.data.gov.sg/v1/transport/carpark-availability?date_time={year}-{month.zfill(2)}-{day.zfill(2)}T{hour.zfill(2)}%3A{min_str.zfill(2)}%3A{sec_str.zfill(2)}'
        t1 = time.time()
        response_API = requests.get(site)
        t2 = time.time()
        response_API.encoding = 'UTF-8'
        raw_data = json.loads(response_API.content)
        t3 = time.time()
        print("req time = ", t2-t1, "json time = ", t3-t2)
        core_list = raw_data['items'][0]['carpark_data']
        return core_list
    except:
        raise Exception()

def json2csv(raw_data, year, month, day, hour, minute):
    header = ['timestamp', 'carpark_number', 'total_lots', 'lots_available', 'lot_type']
    res_df = pd.DataFrame(columns=header)
    timestamp = datetime.datetime(int(year), int(month),int(day))
    timestamp = timestamp.replace(hour=hour, minute=int(minute), second=0)
    for ele in raw_data:
        info = ele['carpark_info'][0]
        res_df = pd.concat([res_df, pd.DataFrame([[timestamp, ele['carpark_number'], info['total_lots'], info['lots_available'], info['lot_type']]], columns=header)])
        # res_df = res_df.append({'timestamp': timestamp, 'carpark_number': ele['carpark_number'], 'total_lots': info['total_lots'], 'lots_available': info['lots_available'], 'lot_type': info['lot_type']}, ignore_index=True)
    return res_df

In [3]:
import threading
class Worker(threading.Thread):
    def __init__(self, year, month, day):
        super().__init__()
        self.year = year
        self.month = month
        self.day = day

    def run(self):
        raw_datalist = []
        minutes = ['00', "15", '30', '45']
        for hour in range(0, 24):
            print(f"start {self.year} {self.month} {self.day} {hour}")
            for mins in minutes:
                try:
                    raw_data = getDataAtTimestamp(self.year, self.month, self.day, str(hour), mins)
                except:
                    print("error in day: " + self.day + " hour: " + str(hour))
                    continue
                raw_df = json2csv(raw_data, self.year, self.month, self.day, hour, mins)
                raw_datalist.append(raw_df)
            print(f"done {self.year} {self.month} {self.day} {hour}")
        raw_datalist = pd.concat(raw_datalist)
        raw_datalist.to_csv(f'./data/{self.year}-{self.month}-{self.day}.csv', index=False)

In [8]:
year = "2022"
month = "8"
workers = []
thread_count = 5
for rg in range(27, 32, thread_count):
    workers = []
    for day in range(rg, rg+thread_count):
        if day > 31:
            continue
        worker = Worker(year, month, str(day))
        worker.start()
        workers.append(worker)
    for worker in workers:
        worker.join()

start 2022 8 27 0
start 2022 8 28 0
start 2022 8 29 0
start 2022 8 30 0
start 2022 8 31 0
req time =  3.005553960800171 json time =  0.0037801265716552734
req time =  2.0773937702178955 json time =  0.0029311180114746094
req time = req time =  0.7633609771728516 json time =  8.821487426757812e-06
 0.7648599147796631 json time =  2.3126602172851562e-05
req time =  0.9021689891815186 json time =  2.002716064453125e-05
req time =  1.2688822746276855 json time =  1.4781951904296875e-05
req time =  1.2661662101745605 json time =  6.9141387939453125e-06
done 2022 8 27 0
start 2022 8 27 1
req time =  2.0235891342163086 json time =  0.07329106330871582
req time =  2.7924437522888184 json time =  0.0035562515258789062
done 2022 8 30 0
start 2022 8 30 1
done 2022 8 29 0
start 2022 8 29 1
req time =  2.018249034881592 json time =  0.003259897232055664
done 2022 8 28 0
start 2022 8 28 1
req time =  2.9927608966827393 json time =  0.0029821395874023438
req time =  1.9603610038757324 json time =  0.

In [10]:
year = "2022"
month = "7"
i = 0
mins = ['00', "15", '30', '45']
for day in range(24,25):
    raw_datalist = []
    for hour in range(24):
        for min in mins:
            try:
                core_list = getDataAtTimestamp(year, month, str(day+1), str(hour), min)
            except:
                continue
            raw_df = json2csv(core_list, year, month, day+1, hour, min)
            raw_datalist.append(raw_df)
        print("done day: " + str(day+1) + " hour: " + str(hour))
    print("done day " + str(day+1))
    raw_datalist = pd.concat(raw_datalist)
    raw_datalist.to_csv(f'./data/{year}-{month}-{day+1}.csv', index=False)


req time =  0.5017690658569336 json time =  8.511543273925781e-05
req time =  0.2451467514038086 json time =  2.9087066650390625e-05
req time =  0.22533679008483887 json time =  2.8133392333984375e-05
req time =  0.837909460067749 json time =  0.0051267147064208984
req time =  0.9719748497009277 json time =  0.003214120864868164
req time =  1.0994670391082764 json time =  0.007214784622192383
done day: 25 hour: 0
req time =  1.0940699577331543 json time =  0.002850055694580078
req time =  1.4018962383270264 json time =  0.003468036651611328
req time =  1.2654249668121338 json time =  0.0033960342407226562
req time =  1.8056621551513672 json time =  0.003106832504272461
done day: 25 hour: 1
req time =  2.3293721675872803 json time =  0.008176803588867188
req time =  1.2151050567626953 json time =  0.007945060729980469
req time =  1.743438959121704 json time =  0.0046041011810302734
req time =  1.7602529525756836 json time =  0.006286144256591797
done day: 25 hour: 2
req time =  0.985427

In [ ]:
df = pd.DataFrame()
for i in range(1,31):
    sub = pd.read_csv(f'{str(i)}.csv')
    df = pd.concat([df, sub])
agg_df = df.groupby('timestamp').sum()
agg_df.head()

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)
ax.plot(agg_df.index, agg_df['lots_available'])
ax.set_xticks(ax.get_xticks()[::24])
plt.xticks(rotation=90)
ax.set_ylabel('Available lots')
ax.set_title('Available lots over Sept 2022')

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(111)
ax.plot(agg_df.index[:24], agg_df['lots_available'][:24])
ax.set_xlabel('Time')
plt.xticks(rotation=90)
ax.set_ylabel('Available lots')
ax.set_title('Available lots over a particular day')

As the plot shows, there is a strong periodic pattern over weekdays and weekends within a week. Within each day, there is also a strong periodic patttern over the days and night, whereby the carpark slots are more available at daytime when people goes to work. This corroborates with our intuition.

# Q1.5

In [ ]:
df = pd.DataFrame()
for i in range(1,31):
    sub = pd.read_csv(f'{str(i)}.csv')
    df = pd.concat([df, sub])
df['available_percentage'] = df['lots_available']/df['total_lots']

In [ ]:
import queue
top5 = queue.PriorityQueue()
unique_keys = set()
ids = df['carpark_number'].unique()
print(len(ids))
for id1 in ids:
    sub1 = df[df['carpark_number'] == id1][['timestamp', 'available_percentage']]
    sub1.drop_duplicates(subset=['timestamp'], inplace=True)
    sub1.index = sub1['timestamp']
    for id2 in ids:
        key = (id1, id2) if id1 < id2 else (id2, id1)
        if id1 == id2 or key in unique_keys:
            continue
        sub2 = df[df['carpark_number'] == id2][['timestamp', 'available_percentage']]
        sub2.drop_duplicates(subset=['timestamp'], inplace=True)
        sub2.index = sub2['timestamp']
        sub2 = sub2.reindex(sub1.index, method='ffill')
        corr = sub1['available_percentage'].corr(sub2['available_percentage'])
        print(f"{id1} and {id2} has correlation {corr}")
        if top5.qsize() < 5:
            top5.put((corr, key))
            unique_keys.add(key)
        elif top5.queue[0][0] < corr:
            removed = top5.get()
            unique_keys.remove(removed[1])
            top5.put((corr, key))
            unique_keys.add(key)